# Выбор локации для скважины

Мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
df1 = pd.read_csv('geo_data_0.csv')
df2 = pd.read_csv('geo_data_1.csv')
df3 = pd.read_csv('geo_data_2.csv')

print(df1.head(5)) # просмотрим содержимое данных
print(df2.head(5))
print(df3.head(5))
df1.info() # информация говорит о том что все данные полные и без пропусков, все типы данных коррекнтые
df2.info()
df3.info()

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to

Данные готовы для исследования. Сначала построим модели обучения для целевого значения product - объём запасов в скважине (тыс. баррелей), мы будем решать залачу регрессии с помощью модели линейной регрессии.

## Обучение и проверка модели

In [3]:
def linarReg(df):
    target = df['product']
    features = df.drop(['product','id'], axis=1)

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train) 

    predictions_valid = model.predict(features_valid)
    predictions_valid_mean = predictions_valid.mean()

    result = mean_squared_error(target_valid, predictions_valid)**0.5
    return result,predictions_valid,target_valid,predictions_valid_mean

In [4]:
result_1,predictions_valid_1,target_valid_1,predictions_valid_mean_1 = linarReg(df1)
result_2,predictions_valid_2,target_valid_2,predictions_valid_mean_2 = linarReg(df2)
result_3,predictions_valid_3,target_valid_3,predictions_valid_mean_3 = linarReg(df3)

print('Регион 1:')
print("RMSE модели линейной регрессии на валидационной выборке:", result_1)
print("Cредний запас предсказанного сырья:", predictions_valid_mean_1)
print('-------------------------------------------------------------------------')
print('Регион 2:')
print("RMSE модели линейной регрессии на валидационной выборке:", result_2)
print("Cредний запас предсказанного сырья:", predictions_valid_mean_2)
print('-------------------------------------------------------------------------')
print('Регион 3:')
print("RMSE модели линейной регрессии на валидационной выборке:", result_3)
print("Cредний запас предсказанного сырья:", predictions_valid_mean_3)

Регион 1:
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
Cредний запас предсказанного сырья: 92.59256778438035
-------------------------------------------------------------------------
Регион 2:
RMSE модели линейной регрессии на валидационной выборке: 0.8930992867756167
Cредний запас предсказанного сырья: 68.728546895446
-------------------------------------------------------------------------
Регион 3:
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
Cредний запас предсказанного сырья: 94.96504596800489


### Промежуточный вывод:

Для каждого региона мы построили модели обучения и установили следующие результаты для регионов:

 - Регион 1:
RMSE модели линейной регрессии на валидационной выборке: 37.6.
Cредний запас предсказанного сырья: 92.6 тыс. баррелей

 - Регион 2:
RMSE модели линейной регрессии на валидационной выборке: 0.89.
Cредний запас предсказанного сырья: 68.7 тыс. баррелей

 - Регион 3:
RMSE модели линейной регрессии на валидационной выборке: 40.
Cредний запас предсказанного сырья: 95 тыс. баррелей

Точнее всего модель предсказывает значения для Регина 2, но в нем же меньше всего средний запас сырья. 

## Подготовка к расчёту прибыли

In [5]:
OIL_PRICE = 450000 # цена нефти за 1 тыс баррелей
BUDGET_WORK = 10_000_000_000 # бюджет на разработку скважин в регионе
BEST_WELLS = 200 # по условию задачи мы выберм 200 лучших скважин
PAYBACK = (BUDGET_WORK/OIL_PRICE)/BEST_WELLS # значение когда наступает окупаемость для 200 лучших скважин в регионе
RANDOM_WELLS = 500
BOOTSTRAP_SAMPLES = 1000
print("Окупаемость месторождений в регионе в тыс баррелей:", PAYBACK)

Окупаемость месторождений в регионе в тыс баррелей: 111.11111111111111


Мы видим что минимальное среднее значение для окупаемости в регионе составляет 111.1 тыс. баррелей. Что является бОльшим числом, чем средний запас предсказанного сырья для всех местророждений в регионах. Это означает, что нам нужно будет выбрать лучшие местрождения для достижения окупаемости. 

## Расчёт прибыли и рисков 

Напишмем функцию для рассчета прибыли по скважинам для каждого региона.

In [6]:
def best_wells_count(target_valid, predictions_valid, count):
    predictions_valid = pd.Series(predictions_valid) # преобразуем предсказания в серию для сортировки
    predictions_sorted = predictions_valid.sort_values(ascending=False) # топ лучших скважин по убыванию
    target_selected = target_valid.reset_index(drop=True)[predictions_sorted.index][:count] # берем истиные значения отобранных скважин
    
    return target_selected.sum() * OIL_PRICE - BUDGET_WORK # суммируем отобранные скважины и перемножаем на цена за тыс. баррелей минус бюджет

In [7]:
print("Прибыль Региона 1 из топ 200 скважин:", best_wells_count(target_valid_1,predictions_valid_1,BEST_WELLS))
print("Прибыль Региона 2 из топ 200 скважин:", best_wells_count(target_valid_2,predictions_valid_2,BEST_WELLS))
print("Прибыль Региона 3 из топ 200 скважин:", best_wells_count(target_valid_3,predictions_valid_3,BEST_WELLS))

Прибыль Региона 1 из топ 200 скважин: 3320826043.1398506
Прибыль Региона 2 из топ 200 скважин: 2415086696.681511
Прибыль Региона 3 из топ 200 скважин: 2710349963.5998325


Как итог каждый из регионов обещает принести прибыль. Но лучший результат прогнозируемой прибыли показывает Регион 1.

Теперь изучим вариант, исходя из условий задачи, что мы берем 500 случайных скважин в регионе, исследуем их и из текущих пятиста выбираем 200 лучших, используя функцию best_wells_count, применяя технику Bootstrap

In [8]:
def revenue(target_valid,predictions_valid,BOOTSTRAP_SAMPLES,RANDOM_WELLS,BEST_WELLS):
    state = np.random.RandomState(12345)
    values = []
    for i in range(BOOTSTRAP_SAMPLES):
        target_sample = target_valid.reset_index(drop=True).sample(RANDOM_WELLS,replace=True,random_state=state)
        probs_subsample = predictions_valid[target_sample.index]
        
        values.append(best_wells_count(target_sample,probs_subsample,BEST_WELLS))
        
    values = pd.Series(values)
    lower = int(values.quantile(0.025))
    upper = values.quantile(0.975)
    risk = (len(values[values<0])/len(values))*100
    return values,lower,upper,risk

In [9]:
values_1,lower_1,upper_1,risk_1 = revenue(target_valid_1,predictions_valid_1,BOOTSTRAP_SAMPLES,RANDOM_WELLS,BEST_WELLS)
values_2,lower_2,upper_2,risk_2 = revenue(target_valid_2,predictions_valid_2,BOOTSTRAP_SAMPLES,RANDOM_WELLS,BEST_WELLS)
values_3,lower_3,upper_3,risk_3 = revenue(target_valid_3,predictions_valid_3,BOOTSTRAP_SAMPLES,RANDOM_WELLS,BEST_WELLS)

print("Средняя прибыль лучших скважин в Регионе 1:",values_1.mean())
print("Доверительный интервал в 95% для средней прибыли в Регионе 1:","от", lower_1,"до", upper_1)
print("Риск убытка в Регионе 1 в %:", risk_1)
print('------------------------------------------------------------------------------------------')
print("Средняя прибыль лучших скважин в Регионе 2:",values_2.mean())
print("Доверительный интервал в 95% для средней прибыли в Регионе 2:","от", lower_2,"до", upper_2)
print("Риск убытка в Регионе 2 в %:", risk_2)
print('------------------------------------------------------------------------------------------')
print("Средняя прибыль лучших скважин в Регионе 3:",values_3.mean())
print("Доверительный интервал в 95% для средней прибыли в Регионе 3:","от", lower_3,"до", upper_3)
print("Риск убытка в Регионе 3 в %:", risk_3)

Средняя прибыль лучших скважин в Регионе 1: 396164984.8023711
Доверительный интервал в 95% для средней прибыли в Регионе 1: от -111215545 до 909766941.5534226
Риск убытка в Регионе 1 в %: 6.9
------------------------------------------------------------------------------------------
Средняя прибыль лучших скважин в Регионе 2: 456045105.7866608
Доверительный интервал в 95% для средней прибыли в Регионе 2: от 33820509 до 852289453.866036
Риск убытка в Регионе 2 в %: 1.5
------------------------------------------------------------------------------------------
Средняя прибыль лучших скважин в Регионе 3: 404403866.5683568
Доверительный интервал в 95% для средней прибыли в Регионе 3: от -163350413 до 950359574.9237995
Риск убытка в Регионе 3 в %: 7.6


### Промежуточный вывод:

Мы составили функцию для рассчетов риска и прибыли. Используя функцию провели исследование по трем регионам применяя технику Bootstrap и получили следуюшие результаты для 200 лучших месторождений в каждом регионе:

 - Средняя прибыль лучших скважин в Регионе 1: 396164985.
Доверительный интервал в 95% для средней прибыли в Регионе 1: от -111215545 до 909766942.
Риск убытка в Регионе 1 в %: 6.9
 - Средняя прибыль лучших скважин в Регионе 2: 456045106.
Доверительный интервал в 95% для средней прибыли в Регионе 2: от 33820509 до 852289454.
Риск убытка в Регионе 2 в %: 1.5
 - Средняя прибыль лучших скважин в Регионе 3: 404403867.
Доверительный интервал в 95% для средней прибыли в Регионе 3: от -163350413 до 950359575.
Риск убытка в Регионе 3 в %: 7.6

Как итог я бы предложил 2 регион для разработки скважин по 2м причинам. 
1) Данные наиболее точны и репрезентативны, так как обученная модель показывает самое лучшее среднее квадратичное отклонение по Региону 2 в RMSE = 0.89, что указывает на высокую точность модели. 

2) Риск получаения убытка в этом регионе меньше 2% в то время как по остальным локациям риск выше 5%.

## Итоговый вывод

Мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

1) Мы провели загрузку и подготовку данных. В нашем распоряжении данные по 3м регионам, в каждом из которых нет пропусков и ошибок. Сначала построим модели обучения для целевого значения product - объём запасов в скважине (тыс. баррелей), мы будем решать залачу регрессии с помощью модели линейной регрессии. Можем приступать к анализу.

2) Обучение и проверка модели. Для каждого региона мы построили модели обучения и установили следующие результаты для регионов:

 - Регион 1:
RMSE модели линейной регрессии на валидационной выборке: 37.6.
Cредний запас предсказанного сырья: 92.6 тыс. баррелей

 - Регион 2:
RMSE модели линейной регрессии на валидационной выборке: 0.89.
Cредний запас предсказанного сырья: 68.7 тыс. баррелей

 - Регион 3:
RMSE модели линейной регрессии на валидационной выборке: 40.
Cредний запас предсказанного сырья: 95 тыс. баррелей

Точнее всего модель предсказывает значения для Регина 2, но в нем же меньше всего средний запас сырья. 

3) Подготовка к расчёту прибыли. Мы установили, что минимальное среднее значение для окупаемости в регионе составляет 111.1 тыс. баррелей. Что является бОльшим числом, чем средний запас предсказанного сырья для всех местророждений в регионах. Это означает, что нам нужно будет выбрать лучшие местрождения для достижения окупаемости.

4) Расчёт прибыли и рисков. Написали функцию для рассчета прибыли по скважинам для каждого региона. Используя функцию провели исследование по трем регионам применяя технику Bootstrap и получили следуюшие результаты для 200 лучших месторождений в каждом регионе:

 - Средняя прибыль лучших скважин в Регионе 1: 396164985.
Доверительный интервал в 95% для средней прибыли в Регионе 1: от -111215545 до 909766942.
Риск убытка в Регионе 1 в %: 6.9
 - Средняя прибыль лучших скважин в Регионе 2: 456045106.
Доверительный интервал в 95% для средней прибыли в Регионе 2: от 33820509 до 852289454.
Риск убытка в Регионе 2 в %: 1.5
 - Средняя прибыль лучших скважин в Регионе 3: 404403867.
Доверительный интервал в 95% для средней прибыли в Регионе 3: от -163350413 до 950359575.
Риск убытка в Регионе 3 в %: 7.6

Как итог я бы предложил 2 регион для разработки скважин по 2м причинам. 
1) Данные наиболее точны и репрезентативны, так как обученная модель показывает самое лучшее среднее квадратичное отклонение по Региону 2 в RMSE = 0.89, что указывает на высокую точность модели. 

2) Риск получаения убытка в этом регионе меньше 2% в то время как по остальным локациям риск выше 5%.